In [1]:
from predictor import tflow_time, torch_time

TensorFlow
conv1d
Model: RandomForestRegressor()
R^2 Score: 0.9790123980640055
Root Mean Squared Error: 0.24798687966908517 ms

conv2d
Model: RandomForestRegressor()
R^2 Score: 0.998552187381041
Root Mean Squared Error: 6.092837144863068 ms

avg1d
Model: RandomForestRegressor()
R^2 Score: 0.9646969669803838
Root Mean Squared Error: 0.07819420130028991 ms

avg2d
Model: RandomForestRegressor()
R^2 Score: 0.9981761127722744
Root Mean Squared Error: 3.7106836448028626 ms

max1d
Model: RandomForestRegressor()
R^2 Score: 0.9866531953049282
Root Mean Squared Error: 0.07893148761416431 ms

max2d
Model: RandomForestRegressor()
R^2 Score: 0.9848965415693467
Root Mean Squared Error: 4.713154512924368 ms

dense
Model: RandomForestRegressor()
R^2 Score: 0.9897332220190377
Root Mean Squared Error: 24.636871454299964 ms

norm1d
Model: RandomForestRegressor()
R^2 Score: 0.942339569673102
Root Mean Squared Error: 0.11165783306039954 ms

norm2d
Model: RandomForestRegressor()
R^2 Score: 0.931945192106844

In [2]:
import tflow_feature_extractor as tf_fx
import torch_feature_extractor as pt_fx

In [3]:
import sys
sys.path.append('../')

In [4]:
import prof_run.tflow_models as TF_Models
import prof_run.torch_models as PT_Models
TF_Model = TF_Models.LeNet5()
tf_model = TF_Model.create()
pt_model = PT_Models.LeNet5().model

batch = 32
channels = 1
epochs = 5
ds = 60000
steps = epochs*ds//batch
dim = 2

In [5]:
numf = 28
tf_feat = tf_fx.extract(tf_model, dim, batch, numf, channels)
numf = 32
pt_feat = pt_fx.extract(pt_model, dim, batch, numf, channels)

In [6]:
tf_feat

[{'name': 'Conv2D',
  'batch': 32,
  'numf': 28,
  'dim': 2,
  'channels': 1,
  'input_shape': TensorShape([32, 28, 28, 1]),
  'output_shape': TensorShape([32, 24, 24, 6]),
  'filters': 6,
  'kernel': (5, 5),
  'stride': (1, 1)},
 {'name': 'tanh',
  'batch': 32,
  'numf': 24,
  'dim': 2,
  'channels': 6,
  'input_shape': TensorShape([32, 24, 24, 6]),
  'output_shape': TensorShape([32, 24, 24, 6])},
 {'name': 'AveragePooling2D',
  'batch': 32,
  'numf': 24,
  'dim': 2,
  'channels': 6,
  'input_shape': TensorShape([32, 24, 24, 6]),
  'output_shape': TensorShape([32, 12, 12, 6]),
  'pool': (2, 2),
  'stride': (2, 2)},
 {'name': 'Conv2D',
  'batch': 32,
  'numf': 12,
  'dim': 2,
  'channels': 6,
  'input_shape': TensorShape([32, 12, 12, 6]),
  'output_shape': TensorShape([32, 8, 8, 16]),
  'filters': 16,
  'kernel': (5, 5),
  'stride': (1, 1)},
 {'name': 'tanh',
  'batch': 32,
  'numf': 8,
  'dim': 2,
  'channels': 16,
  'input_shape': TensorShape([32, 8, 8, 16]),
  'output_shape': Tensor

In [7]:
pt_feat

[{'name': 'Conv2d',
  'input_shape': [32, 1, 32, 32],
  'batch': 32,
  'dim': 2,
  'channels': 1,
  'numf': 32,
  'filters': 6,
  'kernel': (5, 5),
  'stride': (1, 1)},
 {'name': 'Tanh',
  'input_shape': [32, 6, 28, 28],
  'batch': 32,
  'dim': 2,
  'channels': 6,
  'numf': 28},
 {'name': 'AvgPool2d',
  'input_shape': [32, 6, 28, 28],
  'batch': 32,
  'dim': 2,
  'channels': 6,
  'numf': 28,
  'pool': 2,
  'stride': 2},
 {'name': 'Conv2d',
  'input_shape': [32, 6, 14, 14],
  'batch': 32,
  'dim': 2,
  'channels': 6,
  'numf': 14,
  'filters': 16,
  'kernel': (5, 5),
  'stride': (1, 1)},
 {'name': 'Tanh',
  'input_shape': [32, 16, 10, 10],
  'batch': 32,
  'dim': 2,
  'channels': 16,
  'numf': 10},
 {'name': 'AvgPool2d',
  'input_shape': [32, 16, 10, 10],
  'batch': 32,
  'dim': 2,
  'channels': 16,
  'numf': 10,
  'pool': 2,
  'stride': 2},
 {'name': 'Conv2d',
  'input_shape': [32, 16, 5, 5],
  'batch': 32,
  'dim': 2,
  'channels': 16,
  'numf': 5,
  'filters': 120,
  'kernel': (5, 5)

In [8]:
nodes = 3

In [9]:
tf_ms = steps*tflow_time(tf_feat,nodes)/1000

In [10]:
tf_ms/1000

116.11953865371095

In [11]:
pt_ms = steps*torch_time(pt_feat,nodes)/1000

In [12]:
pt_ms/1000

34.89075609082032

In [ ]:
import run_on_nodes as run

In [15]:
parts = ['/home/ubuntu/.env/bin/python3', '/home/ubuntu/prof_run']

In [ ]:
cmd = '/home/ubuntu/.env/bin/python3 /home/ubuntu/prof_run/tflow_lenet.py -nodes {} -batch {} -epochs {}'.format(nodes, batch, epochs)

In [16]:
cmd = '{} {}/tflow_lenet.py -nodes {} -batch {} -epochs {}'.format(parts[0], parts[1], nodes, batch, epochs)

In [17]:
run.go(cmd + ' > tflow_lenet.out 2> tflow_lenet.err', nodes=nodes)

RUNNING CMD:
/home/ubuntu/.env/bin/python3 /home/ubuntu/prof_run/tflow_lenet.py -nodes 3 -batch 32 -epochs 5 > pred.out 2> pred.err
RAN ON FIRST NODE
RAN ON SECOND NODE
RAN ON THIRD NODE
FIRST NODE END
SECOND NODE END
THIRD NODE END


True

In [18]:
cmd = '{} {}/torch_lenet.py -nodes {} -batch {} -epochs {}'.format(parts[0], parts[1], nodes, batch, epochs)

In [19]:
run.go(cmd + ' > torch_lenet.out 2> torch_lenet.err',nodes=nodes)

RUNNING CMD:
/home/ubuntu/.env/bin/python3 /home/ubuntu/prof_run/torch_lenet.py -nodes 3 -batch 32 -epochs 5 > torch_lenet.out 2> torch_lenet.err
RAN ON FIRST NODE
RAN ON SECOND NODE
RAN ON THIRD NODE
FIRST NODE END
SECOND NODE END
THIRD NODE END


True